In [8]:
import os, re, nltk
#nltk.download('stopwords')
from nltk import corpus
from numpy.lib.function_base import vectorize
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import naive_bayes, metrics
# nltk.download('stopwords')

class Email:
    def __init__(self, content, label):
        self.content = content
        self.label = label
        self.preprocessing()

    def preprocessing(self):
        self.content = self.content.lower()
        for word in stopwords.words('english'):
            self.content = re.sub(' ' + word + ' ',  ' ', self.content)
        self.content = re.sub(r'\n', ' ', self.content)
        self.content = re.sub(r'^.+@[^\.].*\.[a-z]{2,}$', 'MailID', self.content)
        self.content = re.sub(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$', 'Links', self.content)
        self.content = re.sub(r'\d+(\.\d+)?', 'numbers', self.content)
        self.content = re.sub(r"[^a-zA-Z0-9]+", " ", self.content)
        self.content = re.sub(r'\s+', ' ', self.content)
        self.content = self.content.strip()

    @staticmethod
    def read():
        if not os.path.exists('messages.csv'):
            return []
        df = pd.read_csv('messages.csv').dropna()
        return list(map(lambda data: Email(data[0], data[1]), zip(df['subject'] + df['message'], df['label'])))

    @staticmethod
    def readTrainTestSplit():
        data = Email.read()
        df = pd.DataFrame()
        df['message'] = [email.content for email in data]
        vector=TfidfVectorizer()
        
        df['label'] = [email.label for email in data]
        return train_test_split(vector.fit_transform(df['message']), df['label'], test_size=0.2, random_state=225, stratify=df['label'])
        

X_train, X_test, Y_train, Y_test = Email.readTrainTestSplit()
X_train=X_train.toarray()
X_test=X_test.toarray()
Y_train=Y_train.to_numpy()
Y_test=Y_test.to_numpy()


In [9]:
#print(X_train.shape)
print(X_test.shape)

(567, 56283)


In [10]:
# Chọn alpha
list_alpha = np.arange(1/100000, 20, 0.1)
score_train = np.zeros(len(list_alpha))
score_test = np.zeros(len(list_alpha))
recall_test = np.zeros(len(list_alpha))
precision_test= np.zeros(len(list_alpha))
count = 0
for alpha in list_alpha:
    bayes = naive_bayes.MultinomialNB(alpha=alpha)
    bayes.fit(X_train, Y_train)
    score_train[count] = bayes.score(X_train, Y_train)
    score_test[count]= bayes.score(X_test, Y_test)
    recall_test[count] = metrics.recall_score(Y_test, bayes.predict(X_test))
    precision_test[count] = metrics.precision_score(Y_test, bayes.predict(X_test))
    count = count + 1 

C:\Users\Test\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Test\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Test\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Test\AppData\Local\Progra

KeyboardInterrupt: 